In [ ]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

!pip install rouge-score nltk --quiet
!pip install huggingface_hub --quiet
!pip install git+https://github.com/google-research/bleurt.git --quiet

     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 47.6 MB/s 
     |████████████████████████████████| 7.6 MB 43.1 MB/s 
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 115 kB 57.8 MB/s 
     |████████████████████████████████| 127 kB 41.4 MB/s 
     |████████████████████████████████| 352 kB 4.7 MB/s 
     |████████████████████████████████| 1.3 MB 60.9 MB/s 


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

<h2> Loading the data and the predictions </h2>



In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

# for local runs
#dataset_path = 'w266_project_data/'
#csv_path = 'w266_project_predictions/'

In [ ]:
# a list of model's output CSV file
output_file_names = ['bart_large_zsl.csv', 'bart_xsum_zsl.csv', 'bart_cnn_zsl.csv', 
                     'bart_large_ft.csv', 'bart_xsum_ft.csv', 'bart_cnn_ft.csv',
                     't5_large_zsl.csv', 't5_large_ft.csv', 'bart_detox_zsl.csv']

In [ ]:
# load the dataset
dataset = load_from_disk(dataset_path)

<h2> ROUGE </h2>

In [ ]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                                 references=df_bart_predictions['test_references'])
  display(output_file_name)
  display(pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()}))
  print()

'bart_large_zsl.csv'

precision    recall  fmeasure
rouge1    0   0.688517  0.781481  0.724142
          1   0.702129  0.794881  0.738238
          2   0.714339  0.808590  0.750571
rouge2    0   0.537471  0.615092  0.567436
          1   0.553202  0.631382  0.583236
          2   0.569334  0.648449  0.599433
rougeL    0   0.684449  0.776822  0.720727
          1   0.697880  0.790671  0.734085
          2   0.711233  0.804450  0.747015
rougeLsum 0   0.684306  0.776621  0.720810
          1   0.697644  0.790556  0.734000
          2   0.711664  0.804500  0.747284

'bart_xsum_zsl.csv'

precision    recall  fmeasure
rouge1    0   0.191019  0.272801  0.212768
          1   0.206122  0.289535  0.227288
          2   0.221542  0.307217  0.243133
rouge2    0   0.089874  0.110479  0.093997
          1   0.102735  0.124492  0.106526
          2   0.117075  0.141416  0.121309
rougeL    0   0.181514  0.256348  0.201172
          1   0.195030  0.272375  0.214562
          2   0.210274  0.288914  0.229320
rougeLsum 0   0.181125  0.256332  0.199828
          1   0.195269  0.272356  0.214597
          2   0.210195  0.288122  0.229614

'bart_cnn_zsl.csv'

...

precision    recall  fmeasure
rouge1    0   0.637967  0.705416  0.656148
          1   0.653627  0.720261  0.670898
          2   0.667829  0.736610  0.684905
rouge2    0   0.486931  0.541429  0.500572
          1   0.503175  0.558681  0.516422
          2   0.520308  0.577213  0.533737
rougeL    0   0.630376  0.696037  0.649057
          1   0.644972  0.711278  0.661979
          2   0.660200  0.727636  0.676845
rougeLsum 0   0.629820  0.694531  0.647661
          1   0.644703  0.711574  0.662236
          2   0.659980  0.728090  0.677463

't5_large_ft.csv'

precision    recall  fmeasure
rouge1    0   0.791790  0.807188  0.793651
          1   0.805683  0.820467  0.806900
          2   0.819684  0.833042  0.819780
rouge2    0   0.673043  0.682798  0.672599
          1   0.690712  0.701534  0.690672
          2   0.708606  0.720530  0.708727
rougeL    0   0.787621  0.802902  0.789588
          1   0.801861  0.816790  0.802980
          2   0.816356  0.830230  0.816767
rougeLsum 0   0.787347  0.803322  0.789701
          1   0.801732  0.816672  0.802820
          2   0.817088  0.830627  0.817647

'bart_detox_zsl.csv'

precision    recall  fmeasure
rouge1    0   0.844398  0.841615  0.838008
          1   0.857302  0.853718  0.850361
          2   0.869847  0.866073  0.862690
rouge2    0   0.745749  0.740237  0.738328
          1   0.763183  0.758268  0.756373
          2   0.780244  0.775732  0.773688
rougeL    0   0.839524  0.836499  0.832944
          1   0.853720  0.850309  0.846827
          2   0.866368  0.862831  0.859450
rougeLsum 0   0.840079  0.836745  0.833743
          1   0.853718  0.850061  0.846943
          2   0.866302  0.863386  0.860000

<h2> Meteor </h2>

In [ ]:
meteor_metric = load_metric("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  meteor_results = meteor_metric.compute(predictions=df_bart_predictions['test_predictions'],
                                 references=df_bart_predictions['test_references'])
  display(output_file_name)
  display(meteor_results['meteor'])
  print()
  # display(pd.concat({k: pd.DataFrame(v) for k, v in meteor_results.items()}))
  # print()

'bart_large_zsl.csv'

0.7831905637834498

'bart_xsum_zsl.csv'

0.3399882588794349

'bart_cnn_zsl.csv'

...

0.7154511639953924

't5_large_ft.csv'

0.8144609259256198

'bart_detox_zsl.csv'

0.8441763640930271

<h2> BLEU </h2>

In [ ]:
bleu_metric = load_metric("bleu")

In [ ]:
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  bleu_predictions = [i.split(' ') for i in df_bart_predictions['test_predictions']]
  bleu_references = [[i.split(' ')] for i in df_bart_predictions['test_references']]

  bleu_results = bleu_metric.compute(predictions=bleu_predictions,
                                 references=bleu_references)
  display(output_file_name)
  bleu_results = pd.DataFrame.from_dict(bleu_results)
  display(bleu_results)
  print()

'bart_large_zsl.csv'

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.375732,0.539229,1.0,1.219567,11331,9291
1,0.375732,0.418004,1.0,1.219567,11331,9291
2,0.375732,0.330161,1.0,1.219567,11331,9291
3,0.375732,0.267814,1.0,1.219567,11331,9291


'bart_xsum_zsl.csv'

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.039412,0.111641,1.0,1.648585,15317,9291
1,0.039412,0.044319,1.0,1.648585,15317,9291
2,0.039412,0.027136,1.0,1.648585,15317,9291
3,0.039412,0.017970,1.0,1.648585,15317,9291


'bart_cnn_zsl.csv'

...

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.318027,0.477364,1.0,1.210096,11243,9291
1,0.318027,0.359079,1.0,1.210096,11243,9291
2,0.318027,0.275523,1.0,1.210096,11243,9291
3,0.318027,0.216598,1.0,1.210096,11243,9291


't5_large_ft.csv'

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.476999,0.625823,1.0,1.079539,10030,9291
1,0.476999,0.515540,1.0,1.079539,10030,9291
2,0.476999,0.431570,1.0,1.079539,10030,9291
3,0.476999,0.371797,1.0,1.079539,10030,9291


'bart_detox_zsl.csv'

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.597164,0.718202,1.0,1.024755,9521,9291
1,0.597164,0.627520,1.0,1.024755,9521,9291
2,0.597164,0.557529,1.0,1.024755,9521,9291
3,0.597164,0.506098,1.0,1.024755,9521,9291


<h2> BLEURT </h2>

In [ ]:
bleurt_metric = load_metric('bleurt')

In [ ]:
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  bleurt_predictions = [i.split(' ') for i in df_bart_predictions['test_predictions']]
  bleurt_references = [[i.split(' ')] for i in df_bart_predictions['test_references']]

  bleurt_results = bleurt_metric.compute(predictions=bleurt_predictions,
                                 references=bleurt_references)
  display(output_file_name)
  bleurt_results = pd.DataFrame.from_dict(bleurt_results).describe()
  display(bleurt_results)
  print()

'bart_large_zsl.csv'

,scores
count,989.000000
mean,-0.035845
std,0.402242
min,-1.265521
25%,-0.331010
50%,-0.017892
75%,0.263814
max,0.880804


'bart_xsum_zsl.csv'

,scores
count,989.000000
mean,-0.764454
std,0.356919
min,-1.634521
25%,-0.977770
50%,-0.847332
75%,-0.652650
max,0.880804


'bart_cnn_zsl.csv'

...

,scores
count,989.000000
mean,-0.164345
std,0.403049
min,-1.287836
25%,-0.451366
50%,-0.169637
75%,0.141818
max,0.880804


't5_large_ft.csv'

,scores
count,989.000000
mean,0.203451
std,0.469670
min,-1.154151
25%,-0.143845
50%,0.266958
75%,0.605917
max,0.926797


'bart_detox_zsl.csv'

,scores
count,989.000000
mean,0.365583
std,0.459727
min,-1.154151
25%,0.035488
50%,0.478290
75%,0.790109
max,0.923983


### examine a few predictions in each model

In [ ]:
print('The first 5 example\'s input and label')
pd.DataFrame(dataset['test'][:5])

The first 5 example's input and label


,en_toxic_comment,en_neutral_comment
0,"holy shit , the 61 61 view must be hilarious .",The 6161 view must be funny
1,all she has to do is smile and keep her cool w...,All she has to do is smile and keep her cool w...
2,can we apply that litmus test to muslims that ...,can we apply that litmus test to muslims that ...
3,"thats fucking it , im joining",That's it! I'm joining.
4,the residents of davis have a collective stick...,The residents of Davis are not so pleasant.


In [ ]:
# examine a few predictions in each model
for output_file_name in output_file_names:
  df_bart_predictions = pd.read_csv(csv_path + output_file_name)
  display(output_file_name)
  display(df_bart_predictions.head(5))
  print()

'bart_large_zsl.csv'

,test_predictions,test_references
0,"['holy shit , the 61 61 view must be hilarious...",['The 6161 view must be funny']
1,['all she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,"['thats fucking it , im joining']","[""That's it! I'm joining.""]"
4,['the residents of davis have a collective sti...,['The residents of Davis are not so pleasant.']


'bart_xsum_zsl.csv'

,test_predictions,test_references
0,['The BBC has released a video of the moment t...,['The 6161 view must be funny']
1,['Theresa May has been praised for the way she...,['All she has to do is smile and keep her cool...
2,['A few years ago I wrote a post saying that i...,['can we apply that litmus test to muslims tha...
3,"[""Thats what I thought about it, so I'm going ...","[""That's it! I'm joining.""]"
4,"[""The BBC has been given a special report on t...",['The residents of Davis are not so pleasant.']


'bart_cnn_zsl.csv'

...

,test_predictions,test_references
0,['the 61 61 view must be hilarious .'],['The 6161 view must be funny']
1,['all she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,['im joining youtube . im joining youtube . im...,"[""That's it! I'm joining.""]"
4,['residents of davis have a collective stick u...,['The residents of Davis are not so pleasant.']


't5_large_ft.csv'

,test_predictions,test_references
0,['The 61 61 view must be hilarious.'],['The 6161 view must be funny']
1,['All she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,"['thats it , im joining']","[""That's it! I'm joining.""]"
4,['The residents of davis have a collective pro...,['The residents of Davis are not so pleasant.']


'bart_detox_zsl.csv'

,test_predictions,test_references
0,['The 61 61 view must be hilarious'],['The 6161 view must be funny']
1,['All she has to do is smile and keep her cool...,['All she has to do is smile and keep her cool...
2,['can we apply that litmus test to muslims tha...,['can we apply that litmus test to muslims tha...
3,"['thats it , im joining']","[""That's it! I'm joining.""]"
4,['The residents of Davis are not good.'],['The residents of Davis are not so pleasant.']
